In [1]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.11.0+cpu  Device: cpu


In [2]:
import os
path_to_folder = '.\data' # путь к папке с данными
name_folders = [x[0].split('\\')[-1] for x in os.walk(path_to_folder)]


In [3]:
name_folders = name_folders[1:]

In [4]:
name_folders

['(',
 ')',
 '+',
 ',',
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '1',
 '9',
 'h',
 't',
 'times',
 'w',
 'X',
 'y']

In [5]:
len(name_folders)

22

In [6]:
NUM_CLASSES = len(name_folders)
MAX_FILES = 1200
MIN_FILES = 1000

In [7]:
class_idx = [i for i in range(len(name_folders))]

In [8]:
dict_folders = {name_folders[i]:class_idx[i] for i in range(len(class_idx))}

In [9]:
dict_folders

{'(': 0,
 ')': 1,
 '+': 2,
 ',': 3,
 '-': 4,
 '0': 5,
 '1': 14,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 15,
 'h': 16,
 't': 17,
 'times': 18,
 'w': 19,
 'X': 20,
 'y': 21}

In [10]:
file_names = []
class_labels = []
for path, subdirs, files in os.walk(path_to_folder):
    for name in files:
        file_names.append(os.path.join(path, name))
        class_labels.append(dict_folders[path.split('\\')[-1]])

In [11]:
import pandas as pd
import skimage
from skimage import io,morphology
from PIL import Image, ImageOps
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset
from torchvision import transforms
from random import shuffle
import cv2 

class NumericDataset(Dataset):

    def __init__(self, root_dir, img_size, num_classes, transform = None):
        self.root_dir = root_dir
        self.img_size = img_size # !!! внимательно при изменении размеров изображения
        self.transform = transform
        self.num_classes = num_classes
        file_names = []
        class_labels = []
        for path, subdirs, files in os.walk(path_to_folder):
            for (idx,name) in enumerate(files):
                if(idx < MAX_FILES and idx > MIN_FILES):
                    file_names.append(os.path.join(path, name))
                    class_labels.append(dict_folders[path.split('\\')[-1]])
        self.files = [[file_names[i],class_labels[i]] for i in range(len(file_names))]#!!!
        shuffle(self.files)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx][0]

        img = cv2.imread(img_name, cv2.COLOR_RGB2GRAY)
        image = img
        #kernel = np.ones((5,5),np.uint8)
        #image = 255 - img #inverting
        #image = cv2.erode(img,kernel,iterations = 1)
        #image = Image.open(img_name)
        image = cv2.resize(image,(self.img_size, self.img_size))
        image = cv2.bitwise_not(image) / 255
        #image[image == 0] = 1
        #image[image==0] = -1
        #image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        image = np.asarray(image).astype(float)#.reshape(3,self.img_size,self.img_size)
        #image = np.dstack([image,image,image])

        #image =  (1 - (torch.from_numpy(image)/255.0)) * 2 -1 

        target = [0 for i in range(self.num_classes)]
        target[self.files[idx][1]] = 1
        target = torch.FloatTensor(target)
        
        image = torch.FloatTensor(image[:,:,0])
        if self.transform:
            image = self.transform(image)

        return image,target,self.files[idx][1]

In [12]:
from torch.utils.data import DataLoader
from torchvision import transforms
#transforms = torch.nn.Sequential(
#    transforms.Resize((28,28))
#)
batch_size = (MAX_FILES-MIN_FILES)*NUM_CLASSES
IMAGE_SIZE = 32
num_train_dataloader = DataLoader(NumericDataset(path_to_folder,IMAGE_SIZE,NUM_CLASSES), batch_size=batch_size, shuffle=True)

In [13]:
for (X_train, y_train,class_idx) in num_train_dataloader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([3781, 32, 32]) type: torch.FloatTensor
y_train: torch.Size([3781, 22]) type: torch.FloatTensor


In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(IMAGE_SIZE*IMAGE_SIZE, 4*IMAGE_SIZE*IMAGE_SIZE)
        #self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(4*IMAGE_SIZE*IMAGE_SIZE, 4*NUM_CLASSES)
        #self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(4*NUM_CLASSES, NUM_CLASSES)

    def forward(self, x):
        x = x.view(-1, IMAGE_SIZE*IMAGE_SIZE)
        #x = F.sigmoid(self.fc1(x))
        x = self.fc1(x)
        #x = self.fc1_drop(x)
        #x = F.sigmoid(self.fc2(x))
        x = self.fc2(x)
        #x = self.fc2_drop(x)
        #return F.softmax(self.fc3(x), dim=1)
        return self.fc3(x)


In [15]:
my_model = Net().to(device)
accs = []
for i in range(20,1001,20):
    my_model.load_state_dict(torch.load('./mlp_model_22_classes_1000/mlp_model_22_classes_'+str(i)+'.pth'))
    my_model.eval()
    acc = 0
    N = 0
    for i in range(len(X_train)):
        result=  my_model(X_train[i])
        N += 1
        if(torch.argmax(y_train[i])==torch.argmax(result)):
            acc += 1   
    acc /=N
    accs.append(acc)
    print(acc)
    

0.8727849775191748
0.87595874107379
0.8704046548532134
0.87595874107379
0.8722560169267389
0.871991536630521
0.871991536630521
0.871991536630521
0.8730494578153928
0.8733139381116107
0.8711980957418672
0.8688177730759058
0.8680243321872521
0.86828881248347
0.868553292779688


KeyboardInterrupt: 

In [ ]:
plt.plot(accs)

In [94]:
accs.index(max(accs))

1

In [95]:
max(accs)

0.89